### Libraries

%pip install gensim nltk pandas sklearn torch rank_bm25

Code based on https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

In [1]:
import json
import pandas as pd
import numpy as np
import multiprocessing
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
import collections

from rank_bm25 import BM25Okapi

# Garbage Collector to fix memory issues
import gc

# 1. Preprocess the data

In [2]:
# Read train claims
with open('../data/train-claims.json', 'r') as f:
    claims = json.load(f)

# Read dev claims
with open('../data/dev-claims.json', 'r') as f:
    dev_claims = json.load(f)

In [3]:
# Lowercasing the 'claim_text' field for each claim
for claim_id, claim_info in claims.items():
    claim_info['claim_text'] = claim_info['claim_text'].lower()

for claim_id, claim_info in dev_claims.items():
    claim_info['claim_text'] = claim_info['claim_text'].lower()

In [4]:
# Read evidence
with open('../data/evidence.json', 'r') as f:
    evidences = json.load(f)

In [5]:
evidences = {i: str.lower(j) for i,j in evidences.items()}

In [6]:
print("Number of claims for training = {0}".format(len(claims)))
print("Number of claims for development = {0}".format(len(dev_claims)))
print("Number of evidences = {0}".format(len(evidences)))

Number of claims for training = 1228
Number of claims for development = 154
Number of evidences = 1208827


## Prepare the corpus

### Second approach

In [7]:
# Collect all texts from claims
corpus = {}

for id, claim in claims.items():
    text2 = claim['claim_text']
    # Create pairs claim + evidence
    for evidence in claim['evidences']:
        text = claim['claim_text'] + " " + evidences[evidence]
        corpus[id + ' - ' + evidence] = (str.strip(text),id)
        text2 = text2 + " " + evidences[evidence]
    # Create pairs claim + all_evidence
    corpus[id] = (str.strip(text2),claim['claim_label'])

for id, evidence in evidences.items():
    corpus[id] = (str.strip(evidence),id) # Add evidence text

### First approach

In [8]:
# Train the model in claims and evidences
# Collect all texts from claims
#corpus = {}
#for id, claim in claims.items():
#    corpus[id] = str.strip(claim['claim_text'])  # Add claim text
#for id, evidence in evidences.items():
#    corpus[id] = str.strip(evidence) # Add evidence text

In [9]:
# 
def tokenize_text(df, column):
    df['tokens'] = df[column].apply(lambda x: [token for token in word_tokenize(x) if token.isalnum()])
    return df

In [10]:
def process_row(row, index):
    return TaggedDocument(row['tokens'], tags=[row[index]])

In [11]:
# Convert the list of documents into a pandas DataFrame
df = pd.DataFrame.from_dict(corpus, orient='index', columns=['text','label'])
df = tokenize_text(df,'text')
df['tagged'] = df.apply(lambda row: process_row(row, 'label'), axis=1)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1214177 entries, claim-1937 - evidence-442946 to evidence-1208826
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1214177 non-null  object
 1   label   1214177 non-null  object
 2   tokens  1214177 non-null  object
 3   tagged  1214177 non-null  object
dtypes: object(4)
memory usage: 46.3+ MB


,text,label,tokens,tagged
claim-1937 - evidence-442946,not only is there no scientific evidence that ...,claim-1937,"[not, only, is, there, no, scientific, evidenc...","([not, only, is, there, no, scientific, eviden..."
claim-1937 - evidence-1194317,not only is there no scientific evidence that ...,claim-1937,"[not, only, is, there, no, scientific, evidenc...","([not, only, is, there, no, scientific, eviden..."
claim-1937 - evidence-12171,not only is there no scientific evidence that ...,claim-1937,"[not, only, is, there, no, scientific, evidenc...","([not, only, is, there, no, scientific, eviden..."
claim-1937,not only is there no scientific evidence that ...,DISPUTED,"[not, only, is, there, no, scientific, evidenc...","([not, only, is, there, no, scientific, eviden..."
claim-126 - evidence-338219,el niño drove record highs in global temperatu...,claim-126,"[el, niño, drove, record, highs, in, global, t...","([el, niño, drove, record, highs, in, global, ..."


In [12]:
train_corpus = df.tagged.values
del df
gc.collect()
train_corpus

array([TaggedDocument(words=['not', 'only', 'is', 'there', 'no', 'scientific', 'evidence', 'that', 'co2', 'is', 'a', 'pollutant', 'higher', 'co2', 'concentrations', 'actually', 'help', 'ecosystems', 'support', 'more', 'plant', 'and', 'animal', 'life', 'at', 'very', 'high', 'concentrations', '100', 'times', 'atmospheric', 'concentration', 'or', 'greater', 'carbon', 'dioxide', 'can', 'be', 'toxic', 'to', 'animal', 'life', 'so', 'raising', 'the', 'concentration', 'to', 'ppm', '1', 'or', 'higher', 'for', 'several', 'hours', 'will', 'eliminate', 'pests', 'such', 'as', 'whiteflies', 'and', 'spider', 'mites', 'in', 'a', 'greenhouse'], tags=['claim-1937']),
       TaggedDocument(words=['not', 'only', 'is', 'there', 'no', 'scientific', 'evidence', 'that', 'co2', 'is', 'a', 'pollutant', 'higher', 'co2', 'concentrations', 'actually', 'help', 'ecosystems', 'support', 'more', 'plant', 'and', 'animal', 'life', 'plants', 'can', 'grow', 'as', 'much', 'as', '50', 'percent', 'faster', 'in', 'concentrati

In [13]:
# Collect all texts from dev claims
dv_corpus = {}

for id, claim in dev_claims.items():
    text2 = claim['claim_text']
    # Create pairs claim + evidence
    for evidence in claim['evidences']:
        text = claim['claim_text'] + " " + evidences[evidence]
        dv_corpus[id + ' - ' + evidence] = (str.strip(text),id)
        text2 = text2 + " " + evidences[evidence]
    # Create pairs claim + all_evidence
    dv_corpus[id] = (str.strip(text2),claim['claim_label'])

In [14]:
# Collect all texts from dev claims
dev_df = pd.DataFrame.from_dict(dv_corpus, orient='index', columns=['text','label'])
dev_df = tokenize_text(dev_df,'text')
dev_df['tagged'] = dev_df.apply(lambda row: process_row(row, 'label'), axis=1)
dev_df.info()
dev_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, claim-752 - evidence-67732 to claim-1021
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    645 non-null    object
 1   label   645 non-null    object
 2   tokens  645 non-null    object
 3   tagged  645 non-null    object
dtypes: object(4)
memory usage: 25.2+ KB


,text,label,tokens,tagged
claim-752 - evidence-67732,[south australia] has the most expensive elect...,claim-752,"[south, australia, has, the, most, expensive, ...","([south, australia, has, the, most, expensive,..."
claim-752 - evidence-572512,[south australia] has the most expensive elect...,claim-752,"[south, australia, has, the, most, expensive, ...","([south, australia, has, the, most, expensive,..."
claim-752,[south australia] has the most expensive elect...,SUPPORTS,"[south, australia, has, the, most, expensive, ...","([south, australia, has, the, most, expensive,..."
claim-375 - evidence-996421,when 3 per cent of total annual global emissio...,claim-375,"[when, 3, per, cent, of, total, annual, global...","([when, 3, per, cent, of, total, annual, globa..."
claim-375 - evidence-1080858,when 3 per cent of total annual global emissio...,claim-375,"[when, 3, per, cent, of, total, annual, global...","([when, 3, per, cent, of, total, annual, globa..."


In [15]:
dev_corpus = dev_df.tagged.values
dev_corpus[0:5]
del dev_df
gc.collect()

0

## Train Model
https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

In [16]:
cores = multiprocessing.cpu_count()
cores

20

In [17]:
# Train Doc2Vec model
model = Doc2Vec(dm=1, vector_size=100, window=5, min_count=1, workers=cores, epochs=50)

In [18]:
model.build_vocab(train_corpus)

In [19]:
from datetime import datetime

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0
        self.last_signal = datetime.now()

    def on_epoch_end(self, model):
        t = datetime.now() - self.last_signal
        print("Epoch #{} - Duration: {}".format(self.epoch, t))
        self.epoch += 1
        self.last_signal = datetime.now()

In [20]:
epoch_logger = EpochLogger()
model.train(train_corpus, total_examples=model.corpus_count,  epochs=model.epochs, callbacks=[epoch_logger])

Epoch #0 - Duration: 0:01:25.614273
Epoch #1 - Duration: 0:01:25.618853
Epoch #2 - Duration: 0:01:27.527903
Epoch #3 - Duration: 0:01:26.822355
Epoch #4 - Duration: 0:01:26.571568
Epoch #5 - Duration: 0:01:27.340967
Epoch #6 - Duration: 0:01:27.417400
Epoch #7 - Duration: 0:01:26.667165
Epoch #8 - Duration: 0:01:27.247633
Epoch #9 - Duration: 0:01:27.723882
Epoch #10 - Duration: 0:01:28.603977
Epoch #11 - Duration: 0:01:27.043769
Epoch #12 - Duration: 0:01:27.309179
Epoch #13 - Duration: 0:01:28.594344
Epoch #14 - Duration: 0:01:31.987251
Epoch #15 - Duration: 0:01:27.626736
Epoch #16 - Duration: 0:01:27.835790
Epoch #17 - Duration: 0:01:27.636705
Epoch #18 - Duration: 0:01:27.210743
Epoch #19 - Duration: 0:01:27.216718
Epoch #20 - Duration: 0:01:26.876791
Epoch #21 - Duration: 0:01:29.553064
Epoch #22 - Duration: 0:01:36.572004
Epoch #23 - Duration: 0:01:36.051439
Epoch #24 - Duration: 0:01:33.348528
Epoch #25 - Duration: 0:01:31.431308
Epoch #26 - Duration: 0:01:29.021999
Epoch #27 -

In [ ]:
model.save("Doc2Vec.model")

## Assesing the model

In [ ]:
evidences_df = pd.DataFrame.from_dict(evidences, orient='index', columns=['text'])
evidences_df = tokenize_text(evidences_df,'text')
evidences_df['inferred'] = evidences_df['tokens'].apply(lambda x: model.infer_vector(x))
evidences_df

In [ ]:
evidences_df.to_pickle('evidences_df.pkl')
#evidences_df = pd.read_pickle('evidences_df.pkl')  

In [ ]:
claims_df = pd.DataFrame.from_dict(claims, orient='index')
claims_df = tokenize_text(claims_df,'claim_text')
claims_df['inferred'] = claims_df['tokens'].apply(lambda x: model.infer_vector(x))
claims_df

In [ ]:
claims_df.to_pickle('claims_df.pkl')
#claims_df = pd.read_pickle('claims_df.pkl')  

### Implement BM25 Retrieval

In [ ]:
# Initialize BM25 model
bm25 = BM25Okapi(evidences_df['tokens'].tolist())

In [ ]:
# Calculate BM25 scores for each claim
def calculate_bm25_scores(query_tokens):
    return bm25.get_scores(query_tokens)

In [ ]:
print("Generating the BM25 scores")
# Compute BM25 scores
bm25_scores = claims_df['tokens'].apply(calculate_bm25_scores)

In [ ]:
bm25_scores.to_pickle('bm25_scores.pkl')
#bm25_scores = pd.read_pickle('bm25_scores.pkl')  

In [ ]:
# Extract lists
claim_vectors = claims_df['inferred'].to_list()
evidence_vectors = evidences_df['inferred'].to_list()

In [ ]:
print("Generating the similarities")
# Calculate Doc2Vec similarities
doc2vec_similarities = cosine_similarity(claim_vectors, evidence_vectors)

In [ ]:
with open('doc2vec_similarities.pkl','wb') as f: pickle.dump(doc2vec_similarities, f)
#with open('doc2vec_similarities.pkl','rb') as f: doc2vec_similarities = pickle.load(f)

In [ ]:
# Normalize the lengths
def normalize(scores):
    return (scores - np.min(scores)) / (np.max(scores) - np.min(scores))

In [ ]:
document_lengths = [len(doc) for doc in evidences_df['tokens']]
normalized_lengths = normalize(document_lengths)
p_normalized_lengths = np.array(0.2 * normalized_lengths)

In [ ]:
n_bm25_scores = [normalize(doc) for doc in bm25_scores]

In [ ]:
n_doc2vec_similarities = normalize(doc2vec_similarities)

In [ ]:
p_bm25_scores = np.array([0.4 * doc for doc in n_bm25_scores])

In [ ]:
p_doc2vec_similarities = np.array([0.4 * doc for doc in n_doc2vec_similarities])

In [ ]:
del document_lengths, normalized_lengths
del bm25_scores, n_bm25_scores
del doc2vec_similarities, n_doc2vec_similarities
gc.collect()

In [ ]:
def get_combined_scores(bm25,similarities,lengths):
    # Initialize an array to store the sum results
    scores = []

    # Perform the element-wise addition
    for i in range(len(similarities)):
        sum_result = bm25[i] + similarities[i] + lengths
        scores.append(sum_result)
    
    return scores

### Dev claims

In [ ]:
# Initialize an array to store the sum results
combined_scores = get_combined_scores(p_bm25_scores, p_doc2vec_similarities, p_normalized_lengths)

In [ ]:
del p_bm25_scores, p_doc2vec_similarities
gc.collect()

In [ ]:
# Rank evidences
ranked_index = np.argsort(combined_scores, axis=1)[:, ::-1]

In [ ]:
with open('ranked_index.pkl','wb') as f: pickle.dump(ranked_index, f)
#with open('ranked_index.pkl','rb') as f: ranked_index = pickle.load(f)

In [ ]:
dev_df = pd.DataFrame.from_dict(dev_claims, orient='index')
dev_df = tokenize_text(dev_df,'claim_text')
dev_df['inferred'] = dev_df['tokens'].apply(lambda x: model.infer_vector(x))
dev_df

In [ ]:
print("Generating the dev BM25 scores")
# Compute dev BM25 scores
dev_bm25_scores = dev_df['tokens'].apply(calculate_bm25_scores)
n_dev_bm25_scores = [normalize(doc) for doc in dev_bm25_scores]
p_dev_bm25_scores = np.array([0.4 * doc for doc in n_dev_bm25_scores])

In [ ]:
dev_bm25_scores.to_pickle('dev_bm25_scores.pkl')
#dev_bm25_scores = pd.read_pickle('dev_bm25_scores.pkl')  

In [ ]:
# Extract lists
claim_dev_vectors = dev_df['inferred'].to_list()

In [ ]:
print("Generating the dev similarities")
# Calculate Doc2Vec similarities
dev_doc2vec_similarities = cosine_similarity(claim_dev_vectors, evidence_vectors)
n_dev_doc2vec_similarities = normalize(dev_doc2vec_similarities)
p_dev_doc2vec_similarities = np.array([0.4 * doc for doc in n_dev_doc2vec_similarities])

In [ ]:
with open('dev_doc2vec_similarities.pkl','wb') as f: pickle.dump(dev_doc2vec_similarities, f)
#with open('dev_doc2vec_similarities.pkl','rb') as f: dev_doc2vec_similarities = pickle.load(f)

In [ ]:
# Initialize an array to store the sum results
dev_combined_scores = get_combined_scores(p_dev_bm25_scores, p_dev_doc2vec_similarities, p_normalized_lengths)

In [ ]:
del p_dev_bm25_scores, p_dev_doc2vec_similarities
gc.collect()

In [ ]:
# Rank evidences
dev_ranked_index = np.argsort(dev_combined_scores, axis=1)[:, ::-1]

## Classification

In [ ]:
# Convert the list of documents into a pandas DataFrame
df = pd.DataFrame.from_dict(corpus, orient='index', columns=['text','label'])
df = tokenize_text(df,'text')
df['tagged'] = df.apply(lambda row: process_row(row, 'label'), axis=1)
df = df[~df['label'].str.startswith('claim')]

# Filter rows where the label does not start with 'claim'
train_corpus = df.tagged.values

In [ ]:
# Collect all texts from dev claims
dev_df = pd.DataFrame.from_dict(dv_corpus, orient='index', columns=['text','label'])
dev_df = tokenize_text(dev_df,'text')
dev_df['tagged'] = dev_df.apply(lambda row: process_row(row, 'label'), axis=1)
dev_df = dev_df[~dev_df['label'].str.startswith('claim')]

# Filter rows where the label does not start with 'claim'
dev_corpus = dev_df.tagged.values

In [ ]:
# Label Distribution
fig, axs = plt.subplots(1, 2, figsize=(16,6))

axs[0].set_title("Train")
axs[1].set_title("Validation")
tlabel = axs[0].hist(sorted([l for l in df['label']]))
vlabel = axs[1].hist(sorted([l for l in dev_df['label']]))

In [ ]:
def vec_for_learning(model, sents):
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [ ]:
logreg = LogisticRegression(n_jobs=cores, C=1e5, max_iter=300)

In [ ]:
y_train, X_train = vec_for_learning(model, train_corpus)

In [ ]:
y_dev, X_dev = vec_for_learning(model, dev_corpus)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_dev)
print('Testing accuracy %s' % accuracy_score(y_dev, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_dev, y_pred, average='weighted')))
report = classification_report(y_dev, y_pred)
print(f"Classification Report:\n{report}")

In [ ]:
y_train_pred = logreg.predict(X_train)
print('Testing accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Testing F1 score: {}'.format(f1_score(y_train, y_train_pred, average='weighted')))
report = classification_report(y_train, y_train_pred)
print(f"Classification Report:\n{report}")

In [ ]:
predictions = {}
for i in range(len(claims_df)):
    ev_list = ['evidence-'+ str(num) for num in ranked_index[i][:5] ]
    predictions[claims_df.index[i]] = {}
    predictions[claims_df.index[i]]["claim_text"] = claims_df.claim_text[i]
    predictions[claims_df.index[i]]["claim_label"] = y_train_pred[i]
    predictions[claims_df.index[i]]["evidences"] = ev_list

In [ ]:
# Export the DataFrame to a JSON file
train_df_doc2vec = pd.DataFrame.from_dict(predictions, orient='index') 
train_df_doc2vec.to_json('../data/train_claims_doc2vec.json', orient='index')

In [ ]:
dev_predictions = {}
for i in range(len(dev_df)):
    ev_list = ['evidence-'+ str(num) for num in ranked_index[i][:5] ]
    dev_predictions[dev_df.index[i]] = {}
    dev_predictions[dev_df.index[i]]["claim_text"] = dev_df.text[i]
    dev_predictions[dev_df.index[i]]["claim_label"] = y_pred[i]
    dev_predictions[dev_df.index[i]]["evidences"] = ev_list

In [ ]:
# Export the DataFrame to a JSON file
dev_df_doc2vec = pd.DataFrame.from_dict(dev_predictions, orient='index') 
dev_df_doc2vec.to_json('../data/dev_claims_doc2vec.json', orient='index')

In [ ]:
gc.collect()

Information Retrieval Prediction

In [ ]:
# Load information retrieval
ir_dev_df = pd.read_pickle('dev-trained.pkl')

In [ ]:
# Collect all texts from dev claims
ir_dev_corpus = {}

for i in range(len(ir_dev_df)):
    text2 = ir_dev_df.iloc[i].claim_text
    # Create pairs claim + evidence
    for evidence in ir_dev_df.iloc[i].evidences:
        text2 = text2 + " " + evidences[evidence]
    # Create pairs claim + all_evidence
    ir_dev_corpus[ir_dev_df.iloc[i].name] = (str.strip(text2),ir_dev_df.iloc[i].claim_label)

In [ ]:
# Collect all texts from dev claims
ir_dev_df = pd.DataFrame.from_dict(ir_dev_corpus, orient='index', columns=['text','label'])
ir_dev_df = tokenize_text(ir_dev_df,'text')
ir_dev_df['tagged'] = ir_dev_df.apply(lambda row: process_row(row, 'label'), axis=1)

ir_dev_corpus = ir_dev_df.tagged.values

In [ ]:
y_ir_dev, X_ir_dev = vec_for_learning(model, ir_dev_corpus)

In [ ]:
y_pred = logreg.predict(X_ir_dev)
print('Testing accuracy %s' % accuracy_score(y_ir_dev, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_ir_dev, y_pred, average='weighted')))
report = classification_report(y_ir_dev, y_pred)
print(f"Classification Report:\n{report}")